In [ ]:
%matplotlib inline
import glob
import _init_paths
from fast_rcnn.config import cfg
from utils.timer import Timer
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import caffe, os, sys, cv2
import argparse
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list
from datasets.config import CLASS_SETS
from dev import dual_detect
import time

In [ ]:
def visualization(img_path, outputs):
    """Draw detected bounding boxes."""
    im = cv2.imread(img_path)
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(im, aspect='equal')
    
    for output in outputs:
      
        ax.add_patch(
            plt.Rectangle((output['xmin'], output['ymin']),
                          output['xmax'] - output['xmin'],
                          output['ymax'] - output['ymin'], fill=False,
                          edgecolor='red', linewidth=3.5)
            )
        ax.text(output['xmin'], output['ymin'] - 2,
                '{:s} {:.3f} , {:s} {:.3f}'.format(output["class"], output["score"]\
                                    ,output["sub"]['class'], output["sub"]["score"]),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

       
    plt.axis('off')
    plt.tight_layout()
    plt.draw()    

In [ ]:
cfg_from_file("../models/pvanet/cfgs/submit_160715.yml")   
prototxt_main = "../models/pvanet/lite/coco_test.prototxt"
caffemodel_main = "../models/rc1_iter_200000.caffemodel"
if not os.path.isfile(caffemodel_main):
    raise IOError(('{:s} not found').format(caffemodel_main))                    

prototxt_sub = "../models/pvanet/lite/coco_test.prototxt"
caffemodel_sub = "../models/rc1_iter_200000.caffemodel"
if not os.path.isfile(caffemodel_sub):
    raise IOError(('{:s} not found').format(caffemodel_sub))                           


net_main = caffe.Net(prototxt_main, caffemodel_main, caffe.TEST)    
net_sub = caffe.Net(prototxt_sub, caffemodel_sub, caffe.TEST)
GPU_ID = 0
caffe.set_mode_gpu()
caffe.set_device(GPU_ID)
cfg.GPU_ID = GPU_ID 

CLASSES_main = CLASS_SETS['main']
CLASSES_sub = CLASS_SETS['subordinate'] 



In [ ]:
test_imgs = glob.glob("../data/demo/*.jpg")
for test_img in test_imgs:
    tic = time.time()
    outputs = (dual_detect(net_main, net_sub, CLASSES_main, CLASSES_sub, test_img, IOU_THRESH=0.75))
    #print(outputs)
    visualization(test_img, outputs)
    #print(toc-tic)